In [50]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb

import os
PATH = 'F:/dataset'#数据集目录地址
datasets = ['train','test']
train_files = ['NORMAL','B','OR','IR']
train_file_nums = [2,6,14,6]

In [51]:
for i in range(train_file_nums[0]):
    tmp_data = pd.read_csv(os.path.join(PATH, 'train', train_files[0]+str(i+1)+".csv"))
    #tmp_data = pd.read_csv(os.path.join(DATA_DIR_PATH, datasets[0], train_files[0]+"0"+str(i+1)+".csv"))
    #如果没有去掉文件名中的0则应使用该行代码，下同 
    tmp_data['label'] = 0
    tmp_data['file'] = i+1
    if i == 0:
        normal_data = tmp_data
    else:
        normal_data = pd.concat([normal_data, tmp_data], axis = 0)
        
for i in range(train_file_nums[1]):
    tmp_data = pd.read_csv(os.path.join(PATH, 'train', train_files[1]+str(i+1)+".csv"))
    tmp_data['label'] = 1
    tmp_data['file'] = i+1
    if i == 0:
        ball_data = tmp_data
    else:
        ball_data = pd.concat([ball_data, tmp_data], axis = 0)
        
for i in range(train_file_nums[2]):
    if i >= 9:
        filename = train_files[2]+str(i+1)+".csv"
    else:
        filename = train_files[2]+str(i+1)+".csv"
    tmp_data = pd.read_csv(os.path.join(PATH, 'train', filename))
    tmp_data['label'] = 2
    tmp_data['file'] = i+1
    if i == 0:
        or_data = tmp_data
    else:
        or_data = pd.concat([or_data, tmp_data], axis = 0)
        
for i in range(train_file_nums[3]):
    tmp_data = pd.read_csv(os.path.join(PATH,'train', train_files[3]+str(i+1)+".csv"))
    tmp_data['label'] = 3
    tmp_data['file'] = i+1
    if i == 0:
        ir_data = tmp_data
    else:
        ir_data = pd.concat([ir_data, tmp_data], axis = 0)
        
train_data = pd.concat([normal_data,ball_data,or_data,ir_data])

e:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:43: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [52]:
from scipy import stats,signal,fftpack
from pywt import wavedec
"""特征提取
 时域：
 均值，标准差，最大值，最小值，均方根，峰峰值，中位数，四分位差，百分位差，
 偏度，峰度，方差，整流平均值，方根幅值，波形因子，峰值因子，脉冲值，裕度
 频域：
 均值，标准差，最大值，最小值，均方根，中位数，四分位差，百分位差
 f2 f3 f4反映频谱集中程度
 f5 f6 f7 f8反映主频带位置
 5级小波变换，最后输出6个能量特征和其归一化能量特征"""
    
columns = ['time_mean','time_std','time_max','time_min','time_rms','time_ptp','time_median',
                                     'time_iqr','time_pr','time_skew','time_kurtosis','time_var','time_amp',
                                     'time_smr','time_wavefactor','time_peakfactor','time_pulse','time_margin',
                                     'freq_mean','freq_std','freq_max','freq_min','freq_rms','freq_median',
                                     'freq_iqr','freq_pr','freq_f2','freq_f3','freq_f4','freq_f5','freq_f6','freq_f7',
                                     'freq_f8','ener_cA5','ener_cD1','ener_cD2','ener_cD3','ener_cD4','ener_cD5',
                                     'ratio_cA5','ratio_cD1','ratio_cD2','ratio_cD3','ratio_cD4','ratio_cD5']
data_names = ['DE_','FE_']
new_columns = []
for data_name in data_names:
    new_columns.extend([data_name+j for j in columns])


def featuring(this_file_data,windowlen):
    n_line = this_file_data.shape[0]
    features = []
    for i in range (0,n_line,windowlen):
        if (n_line - i)/windowlen >= 1:
            df = this_file_data.loc[i:i+windowlen,['DE_time','FE_time']]
            feature_list = []
            for col in df.columns:
                df_line = df[col]
                time_mean = df_line.mean()                                           
                time_std = df_line.std()
                time_max = df_line.max()
                time_min = df_line.min()
                time_rms = np.sqrt(np.square(df_line).mean())
                time_ptp = time_max-time_min 
                time_median = np.median(df_line)
                time_iqr = np.percentile(df_line,75)-np.percentile(df_line,25)
                time_pr = np.percentile(df_line,90)-np.percentile(df_line,10)
                time_skew = stats.skew(df_line)
                time_kurtosis = stats.kurtosis(df_line)
                time_var = np.var(df_line)
                time_amp = np.abs(df_line).mean()
                time_smr = np.square(np.sqrt(np.abs(df_line)).mean())
#               下面四个特征需要注意分母为0或接近0问题，可能会发生报错
                time_wavefactor = time_rms/time_amp
                time_peakfactor = time_max/time_rms
                time_pulse = time_max/time_amp
                time_margin = time_max/time_smr
#               采样频率25600Hz
                df_fftline = fftpack.fft(df[col])
                freq_fftline = fftpack.fftfreq(len(df[col]),1/25600)
                df_fftline = abs(df_fftline[freq_fftline>=0])
                freq_fftline = freq_fftline[freq_fftline>=0]
                freq_mean = df_fftline.mean()
                freq_std = df_fftline.std()
                freq_max = df_fftline.max()
                freq_min = df_fftline.min()
                freq_rms = np.sqrt(np.square(df_fftline).mean())
                freq_median = np.median(df_fftline)
                freq_iqr = np.percentile(df_fftline,75)-np.percentile(df_fftline,25)
                freq_pr = np.percentile(df_fftline,90)-np.percentile(df_fftline,10)
                freq_f2 = np.square((df_fftline-freq_mean)).sum()/(len(df_fftline)-1)
                freq_f3 = pow((df_fftline-freq_mean),3).sum()/(len(df_fftline)*pow(freq_f2,1.5))
                freq_f4 = pow((df_fftline-freq_mean),4).sum()/(len(df_fftline)*pow(freq_f2,2))
                freq_f5 = np.multiply(freq_fftline,df_fftline).sum()/df_fftline.sum()
                freq_f6 = np.sqrt(np.multiply(np.square(freq_fftline),df_fftline).sum())/df_fftline.sum()
                freq_f7 = np.sqrt(np.multiply(pow(freq_fftline,4),df_fftline).sum())/np.multiply(np.square(freq_fftline),df_fftline).sum()
                freq_f8 = np.multiply(np.square(freq_fftline),df_fftline).sum()/np.sqrt(np.multiply(pow(freq_fftline,4),df_fftline).sum()*df_fftline.sum())
                #----------  timefreq-domain feature,12
                # 5级小波变换，最后输出6个能量特征和其归一化能量特征
                cA5, cD5, cD4, cD3, cD2, cD1 = wavedec(df[col], 'db10', level=5)
                ener_cA5 = np.square(cA5).sum()
                ener_cD5 = np.square(cD5).sum()
                ener_cD4 = np.square(cD4).sum()
                ener_cD3 = np.square(cD3).sum()
                ener_cD2 = np.square(cD2).sum()
                ener_cD1 = np.square(cD1).sum()
                ener = ener_cA5 + ener_cD1 + ener_cD2 + ener_cD3 + ener_cD4 + ener_cD5
                ratio_cA5 = ener_cA5/ener
                ratio_cD5 = ener_cD5/ener
                ratio_cD4 = ener_cD4/ener
                ratio_cD3 = ener_cD3/ener
                ratio_cD2 = ener_cD2/ener
                ratio_cD1 = ener_cD1/ener

                feature_list.extend([time_mean,time_std,time_max,time_min,time_rms,time_ptp,time_median,
                                     time_iqr,time_pr,time_skew,time_kurtosis,time_var,time_amp,
                                     time_smr,time_wavefactor,time_peakfactor,time_pulse,time_margin,
                                     freq_mean,freq_std,freq_max,freq_min,freq_rms,freq_median,
                                     freq_iqr,freq_pr,freq_f2,freq_f3,freq_f4,freq_f5,freq_f6,freq_f7,
                                     freq_f8,ener_cA5,ener_cD1,ener_cD2,ener_cD3,ener_cD4,ener_cD5,
                                     ratio_cA5,ratio_cD1,ratio_cD2,ratio_cD3,ratio_cD4,ratio_cD5])
            features.append(feature_list)
    features = np.asarray(features)
    return features


def Trainfeaturing(train_files,train_data,windowlen):
    flag = 0
    for label in range(len(train_files)):
        this_label_data = train_data[train_data.label == label]
        n_file = max(this_label_data.file)
        for file_index in range(1,n_file+1):
            this_file_data = this_label_data[this_label_data.file == file_index]
            if flag == 0:
                features = featuring(this_file_data,windowlen)
                flag += 1
                labels = [label for j in range(features.shape[0])]
            else:
                new_features = featuring(this_file_data,windowlen)
                features = np.concatenate((features, new_features), axis=0)
                labels.extend([label for j in range(new_features.shape[0])])
    features = np.asarray(features)
    labels = np.asarray(labels)
    return features,labels



In [53]:
test_file = 'TEST'
test_file_num = 142
windowlen = 1750

features,labels = Trainfeaturing(train_files,train_data,windowlen)

In [54]:
columns = ['time_mean','time_std','time_max','time_min','time_rms','time_ptp','time_median',
                                     'time_iqr','time_pr','time_skew','time_kurtosis','time_var','time_amp',
                                     'time_smr','time_wavefactor','time_peakfactor','time_pulse','time_margin',
                                     'freq_mean','freq_std','freq_max','freq_min','freq_rms','freq_median',
                                     'freq_iqr','freq_pr','freq_f2','freq_f3','freq_f4','freq_f5','freq_f6','freq_f7',
                                     'freq_f8','ener_cA5','ener_cD1','ener_cD2','ener_cD3','ener_cD4','ener_cD5',
                                     'ratio_cA5','ratio_cD1','ratio_cD2','ratio_cD3','ratio_cD4','ratio_cD5']
data_names = ['DE_','FE_']
new_columns = []
for data_name in data_names:
    new_columns.extend([data_name+j for j in columns])
fea_table = pd.DataFrame(features,columns=new_columns)
fea_table['label'] = labels
fea_table.to_csv(os.path.join(PATH,'trainresult.csv'),index=False)

In [55]:
from sklearn.model_selection  import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(features, labels, train_size=0.9, random_state=10010)

In [56]:
# 评分规则，f1-score 权重 ball inner outer 各占0.3 normal 0.1
from sklearn.metrics import f1_score,confusion_matrix
def get_final_score(real,predict,mat=False,score=False):
    # 共有4类
    final_score = 0
    if mat:
        mat = confusion_matrix(real,predict)
        print(mat)
    for i in range(4):
        real_i = (real == i)
        # print(real_i)
        predict_i = (predict == i)
        # print(predict_i)
        this_f1_score = f1_score(real_i, predict_i)
        if score:
            print(i,":",this_f1_score)
        if i == 0:
            this_f1_score = this_f1_score*0.1
        else:
            this_f1_score = this_f1_score*0.3
        final_score += this_f1_score
    return final_score

In [57]:
import lightgbm as lgb
params = {  
 'boosting_type': 'gbdt',  
    'objective': 'multiclass',  
    'num_class': 4,  
    'metric': 'multi_error',  
    'num_leaves': 200,  
    'min_data_in_leaf': 40,  
    'learning_rate': 0.05,
    'feature_fraction': 0.85,  
    'bagging_fraction': 0.85,  
    'bagging_freq':10,  
    'lambda_l1': 0.1,  
    'lambda_l2': 0.1,  
    'min_gain_to_split': 0.2,  
    'verbose': -1, 
    'is_unbalance':'true'
}  

print('Training...')
trn_data = lgb.Dataset(X_train, Y_train)

clf = lgb.train(params, 
                trn_data ,
                num_boost_round = 1000,
                #valid_sets = [X_train,X_test], 
                verbose_eval = 100, 
                #early_stopping_rounds = 100
               )


y_prob = clf.predict(X_test, num_iteration=clf.best_iteration)
y_pred = [list(x).index(max(x)) for x in y_prob]
y_pred=np.array(y_pred)

print(Y_test)
print(y_pred)

scores = []

scores.append(get_final_score(Y_test,y_pred))
scores = np.asarray(scores)
print("mean_score",scores.mean())

import joblib
joblib.dump(clf,'cwru.model')

Training...
[3 3 0 0 0 1 0 0 0 0 0 3 0 0 3 0 0 2 1 0 0 1 3 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 3 0 2 2 0 2 0 0 0 3]
[3 3 0 0 0 1 0 0 0 0 0 3 0 0 3 0 0 2 1 0 0 1 3 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 3 0 2 2 0 2 0 0 0 3]
mean_score 1.0


['cwru.model']

In [58]:
m='/test/TEST'#/test为存储测试数据的文件夹名，TEST为文件名数字前的TEST
a=PATH+m
b='.csv'
windowlen=1750

print('Predicting...')
import csv
f = open("result.csv", "w",encoding='utf-8',newline='') 
csv_writer = csv.writer(f)
f.close()
#清空文件
f = open("result.csv", "a",encoding='utf-8',newline='') 
csv_writer = csv.writer(f)
csv_writer.writerow(["filename","label"])
for s in range(1,143):
    print('file',s)
    temp=a+str(s)+b
    temp2="TEST"+str(s)
    params['testdata_path'] = str(temp)
    data7 = pd.DataFrame(pd.read_csv(params['testdata_path']))
    X_test = featuring(data7,1750)
    y_prob = clf.predict(X_test, num_iteration=clf.best_iteration)

    y_pred = [list(x).index(max(x)) for x in y_prob]
    y_pred = str(y_pred)
    
    y_pred=y_pred.replace("[", "");
    y_pred=y_pred.replace("]", "");
    print(co(y_pred))
    csv_writer = csv.writer(f)

    csv_writer.writerow([temp2,co(y_pred)])
f.close()
print("succeed")

Predicting...
file 1
0
file 2
0
file 3
3
file 4
1
file 5
0
file 6
0
file 7
0
file 8
2
file 9
0
file 10
1
file 11
0
file 12
0
file 13
0
file 14
0
file 15
0
file 16
0
file 17
3
file 18
0
file 19
0
file 20
0
file 21
1
file 22
2
file 23
0
file 24
0
file 25
0
file 26
0
file 27
0
file 28
0
file 29
0
file 30
0
file 31
0
file 32
0
file 33
0
file 34
0
file 35
0
file 36
3
file 37
3
file 38
0
file 39
0
file 40
0
file 41
0
file 42
2
file 43
0
file 44
0
file 45
0
file 46
0
file 47
0
file 48
0
file 49
0
file 50
0
file 51
0
file 52
0
file 53
0
file 54
2
file 55
0
file 56
0
file 57
0
file 58
0
file 59
0
file 60
1
file 61
0
file 62
1
file 63
0
file 64
0
file 65
3
file 66
0
file 67
2
file 68
2
file 69
1
file 70
0
file 71
2
file 72
0
file 73
0
file 74
0
file 75
2
file 76
0
file 77
0
file 78
0
file 79
2
file 80
0
file 81
0
file 82
0
file 83
2
file 84
0
file 85
0
file 86
0
file 87
0
file 88
0
file 89
0
file 90
0
file 91
0
file 92
0
file 93
0
file 94
0
file 95
0
file 96
0
file 97
0
file 98
0
file 99
0
file 